# Homework 01

In [17]:
import numpy as np
import numpy.typing as npt

In [18]:
matrix = np.array([[0, 1, -1], [-1, 0, 1], [1, -1, 0]])
row_strategy = np.array([[0.1, 0.2, 0.7]])
column_strategy = np.array([[0.3, 0.2, 0.5]]).transpose()

## 2) Strategy Pair Evaluation

- Given mixed strategies of two players and two utility matrices, compute value for each player.
- Do the same, but for zero-sum games. This time, your function should only take a single matrix (as the other one is implied)

In [19]:
def evaluate(row_strategy: npt.NDArray,
             column_strategy: npt.NDArray,
             row_matrix: npt.NDArray = None,
             column_matrix: npt.NDArray = None,
             matrix: npt.NDArray = None
             ) -> tuple[np.double, np.double]:
    """
    Given matrix of strategies of two players and two utility matrices, compute
    value for each player.

    Args:
        row_strategy: Strategy of the row player
        row_matrix: Utility matrix for the row player
        column_strategy: Strategy for the column player
        column_matrix: Utility matrix for the row player
        matrix: Utility matrix for both players (row player). If this matrix
                is supplied, game is assumed to be zero sum.

    Returns:
        Values (row_value, column_value) of the utility function for both
        players.

    """

    assert (
            row_strategy is not None and
            column_strategy is not None and
            matrix is not None
    )

    # Normalize vector shapes
    row_strategy.reshape(1, -1)
    column_strategy.reshape(-1, 1)

    if matrix is not None:
        row_matrix = matrix
        column_matrix = -matrix

    assert row_matrix is not None or column_matrix is not None
    # Assume game is zero-sum if one matrix is not supplied
    if row_matrix is None:
        row_matrix = -column_matrix
    elif column_matrix is None:
        column_matrix = -row_matrix

    row_value = (row_strategy @ row_matrix @ column_strategy).item()
    column_value = (row_strategy @ column_matrix @ column_strategy).item()

    return row_value, column_value

In [20]:
evaluate(matrix=matrix, column_strategy=column_strategy,
         row_strategy=row_strategy)

(0.07999999999999997, -0.07999999999999997)

# 3) Best Response Calculation

- Given a mixed strategy of a player and a utility matrix, compute opponent's best responding strategy

Let $x \in \Pi_1$ be the strategy of the row player. We want to find $y*_1 \in \Pi_2$ such, that:
$$
\begin{align*}
y^* = \arg\max_{y' \in \Pi_2}{u_1(x, y')}
\end{align*}
$$

Let $A = \mathbb{R}^{|A_1| \times |A_2|}$ be the utility matrix of the column player. We have
\begin{align*}
u_1(x, y) = \sum_{(a_1, a_2) \in a} x(a_1)y(a_2)u_i(a_1, a_2) = x^T A y
\end{align*}

We show that there exists a _pure strategy_ $e_j$ such that
$$
x^T A e_j = x^T A y^*
$$

First notice that surely $x^T A e_j \le x^T A y^*$ for every $j$. Now we show that there must be $j$ such that this inequality holds with equality. For contradiction suppose that this is not the case. Then
$$
    x^T A y^*
    = x^T A \left(\sum_j y_j^* e_j \right)
    = \sum_j y_j^* x^T A  e_j
    < x A y^* \sum_j y_j^*
    = x A y^*
$$
which is a contradiction.


In [21]:
def best_response(matrix, row_strategy=None, column_strategy=None):
    """
    Given a mixed strategy of a player and a utility matrix, compute
    opponent's best responding strategy.

    Note: Assumes game is zero-sum and matrix is given as utility matrix for the
     row player.

    Note: Only one of the row_strategy and column_strategy should be
    specified.

    Args:
        matrix: Utility matrix (for row player).
        row_strategy: Strategy of the row player (default=None).
        column_strategy: Strategy for the column player (default=None).

    Returns:
        Best response strategy of the column player.

    """

    assert row_strategy is not None or column_strategy is not None

    pi_2 = None
    if row_strategy is not None:
        matrix = -matrix.T
        pi_2 = row_strategy.reshape(-1, 1)

    if column_strategy is not None:
        pi_2 = column_strategy.reshape(-1, 1)

    m = matrix.shape[0]
    j = np.argmax(matrix @ pi_2)
    br = np.zeros(m, dtype=np.float64)
    br[j] = 1

    return br

In [22]:
(matrix @ column_strategy).reshape(-1).argmax()

1

In [23]:
br_col = best_response(matrix=matrix, row_strategy=row_strategy)
evaluate(matrix=matrix, column_strategy=br_col, row_strategy=row_strategy)

(-0.6, 0.6)

In [24]:
br_row = best_response(matrix=matrix, column_strategy=column_strategy)
evaluate(matrix=matrix, column_strategy=column_strategy, row_strategy=br_row)

(0.2, -0.2)

# 4) Best Response Value Calculation

- Given a mixed strategy of a player and a utility matrix, compute agent's value when facing best-responding opponent.

In [25]:
def best_response_value_row(matrix, row_strategy):
    """
    Compute the value of the row player when facing best responding player.

    Note: Assumes game is zero-sum and matrix is given as utility matrix for the
     row player.

    Args:
        matrix: Utility matrix.
        row_strategy: Strategy of the row player.

    Returns:
        Value of the row player when facing best responding opponent.

    """

    brc = best_response(matrix=matrix, row_strategy=row_strategy)
    return evaluate(
        matrix=matrix,
        row_strategy=row_strategy,
        column_strategy=brc
    )[0]


def best_response_value_column(matrix, column_strategy):
    """
    Compute the value of the column player when facing best responding player.

    Note: Assumes game is zero-sum and matrix is given as utility matrix for the
     row player.

    Args:
        matrix: Utility matrix.
        column_strategy: Strategy of the column player.

    Returns:
        Value of the column player when facing best responding opponent.

    """

    brr = best_response(matrix=matrix, column_strategy=column_strategy)
    return evaluate(
        matrix=matrix,
        row_strategy=brr,
        column_strategy=column_strategy
    )[1]

In [26]:
best_response_value_row(matrix=matrix, row_strategy=row_strategy)

-0.6

In [27]:
best_response_value_column(matrix=matrix, column_strategy=column_strategy)

-0.2

# 5) Finding Dominated Actions

- Given a matrix, find dominated actions of both row and column player

In [28]:
A = np.array([[13, 1, 7], [4, 3, 6], [-1, 2, 8]])
B = np.array([[3, 4, 3], [1, 3, 2], [9, 8, -1]])
A, B

(array([[13,  1,  7],
        [ 4,  3,  6],
        [-1,  2,  8]]),
 array([[ 3,  4,  3],
        [ 1,  3,  2],
        [ 9,  8, -1]]))

In [29]:
def find_dominated_strategies_row(matrix):
    """ Find dominated actions of row player.

    Args:
        matrix: Utility matrix.

    Returns:
        List of dominated actions.

    """

    ans = []
    m = matrix.shape[0]
    for i in range(m):
        for j in range(m):
            if i == j:
                continue

            if np.all(matrix[i, :] > matrix[j, :]):
                ans.append(j)

    return ans


def find_dominated_strategies_column(matrix):
    """ Find dominated actions of column player.

    Args:
        matrix: Utility matrix.

    Returns:
        List of dominated actions.

    """

    return find_dominated_strategies_row(matrix.T)

In [30]:
find_dominated_strategies_row(matrix=B)

[1]

# 6) Iterated Removal of  Dominated Actions

- Given a matrix, run the "iterated removal of dominated actions" algorithm

In [31]:
def iterated_removal_of_dominant_actions(row_matrix, column_matrix):
    """
    Run the iterated removal of dominated actions algorithm.

    Args:
        row_matrix: Utility matrix of the row player.
        column_matrix: Utility matrix of the column player.

    Returns:
        Two tuple (row_dom, col_dom) consisting of two bool vectors specifying
        which actions are were not eliminated.

    """

    m, n = row_matrix.shape
    row_actions = np.ones(m, dtype=bool)
    col_actions = np.ones(n, dtype=bool)

    row_dom = find_dominated_strategies_row(
        row_matrix[row_actions, :][:, col_actions]
    )
    col_dom = find_dominated_strategies_column(
        column_matrix[row_actions, :][:, col_actions]
    )
    while len(row_dom) > 0 or len(col_dom) > 0:
        row_actions[row_dom] = False
        col_actions[col_dom] = False

        row_dom = find_dominated_strategies_row(
            row_matrix[row_actions, :][:, col_actions]
        )
        col_dom = find_dominated_strategies_column(
            column_matrix[row_actions, :][:, col_actions]
        )

    return row_actions, col_actions


In [32]:
iterated_removal_of_dominant_actions(row_matrix=A, column_matrix=B)

(array([False,  True, False]), array([False,  True, False]))